In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time

In [4]:
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Cache is valid for [29/05/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 mac64] driver in cache 
[WDM] - Driver found in cache [/Users/samstruthers/.wdm/drivers/chromedriver/83.0.4103.39/mac64/chromedriver]


In [6]:
#use this function to grab links so you can find players to pull contract history
def get_position_links(position):
    url = f"https://overthecap.com/contract-history/{str.lower(position).replace(' ' ,'-')}/"
    response = requests.get(url)
    time.sleep(2)
    soup = BeautifulSoup(response.text, 'html.parser')
    players_links =[]
    player = []
    for a in soup.select("a[href*=player]"):
        players_links.append(a['href'])
        player.append(a.getText())
    position_df = pd.DataFrame({"Player Link": players_links,
                   "Player Name": player})
    position_df = position_df.drop_duplicates()
    position_df = position_df.reset_index()
    position_df = position_df.drop(columns = 'index')
    return(position_df)

In [ ]:
#example
get_position_links("Running Back")

In [7]:
#use this function to get contract history for specific player
def get_contract_history(player_link, player_name):
    contracts = []
    players_contracts = []
    links = player_link
    name = player_name
    year = []
    base_salary = []
    cap_number = []
    cappercent = []
    experience = []
    years_experience_counter = 0
    url = f"https://overthecap.com/{player_link}"
    player_response = requests.get(url)
    player_soup = BeautifulSoup(player_response.text, 'html.parser')
    table = player_soup.find('table',class_="contract salary-cap-history player-new")
    headers = [th.getText() for th in table.findAll('tr', limit=2)[0].findAll('th')]
    rows = table.findAll('tr')[1:]
    stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    for x in range(len(stats)):
        if stats[x][0] != "Total":
            players_contracts.append(name)
            contracts.append(stats[x])
            years_experience_counter = years_experience_counter + 1
            experience.append(years_experience_counter)
        else:
            continue
        for y in range(len(headers)):               
            if ((stats[x][0] != "Total")&(headers[y] == "Year")):
                year.append(stats[x][y])
            elif (stats[x][0] != "Total")&(headers[y] == "CapNumber"):
                cap_number.append(stats[x][y])
            elif (stats[x][0] != "Total")&(headers[y] == "Base Salary"):
                base_salary.append(stats[x][y])
            elif (stats[x][0] != "Total")&(headers[y] == "Cap %"):
                cappercent.append(stats[x][y])  
            else:
                continue
    years_experience_counter = 0
    df = pd.DataFrame({"Player":players_contracts,
                   'Year':year,
                   "Base Salary":base_salary,
              "Cap Number":cap_number,
             "% of Cap":cappercent,
              "Years of Experience":experience
             })
    return(df)

In [ ]:
#example
get_contract_history("/player/barry-sanders/7729/","Barry Sanders")

In [8]:
#use this function to get a teams current salary cap information
def get_team_contracts(team):
    url = f"https://overthecap.com/salary-cap/{str.lower(team).replace(' ','-')}/"
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table',class_="salary-cap-table contracted-players")
    table
    headers = [th.getText() for th in table.findAll('tr', limit=2)[0].findAll('th')][:11]
    rows = table.findAll('tr')[2:]
    stats = [[td.getText() for td in rows[i].findAll('td')][:11]
            for i in range(len(rows))]
    df = pd.DataFrame(stats, columns = headers)
    return(df)

In [ ]:
#example
get_team_contracts("Los Angeles Chargers")

In [9]:
#use this function to see most current contracts and what would be save if cut/traded
def get_transaction_table():
    url = "http://overthecap.com/transactions-table/"
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table',class_="transactions-table sortable")
    table
    headers = [th.getText() for th in table.findAll('tr', limit=2)[0].findAll('th')]
    rows = table.findAll('tr')[1:]
    stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    df = pd.DataFrame(stats, columns = headers)
    return(df)

In [ ]:
#example
get_transaction_table()